In [1]:
import json
import time

In [2]:
context_corpus = json.load(open('dataset/paragraph_context.json', 'r', encoding='utf8')) 
question_corpus = json.load(open('dataset/question_context.json', 'r', encoding='utf8'))
train_labels_json = json.load(open('dataset/train_labels.json', 'r', encoding='utf8'))
test_labels_json = json.load(open('dataset/test_labels.json', 'r', encoding='utf8'))

In [3]:
from rank_bm25 import BM25Okapi

In [4]:
test_document = [context_corpus[t] for t in test_labels_json]
test_question = [question_corpus[t] for key in test_labels_json.keys() for t in test_labels_json[key]]

In [5]:
from konlpy.tag import Okt, Komoran

In [6]:
st = time.time()
bm25 = BM25Okapi([doc.split() for doc in test_document])
print(time.time() - st)

2.208296298980713


In [8]:
start_time = time.time()
answer = []
for q in [question.split() for question in test_question]:
    answer.append(bm25.get_top_n(q, test_document, n=150))

end_time = time.time()

In [9]:
end_time - start_time

3692.6343388557434

In [11]:
test_question_ids = [t for key in test_labels_json.keys() for t in test_labels_json[key]]

In [13]:
test_document_ids = {context_corpus[t]:t for t in test_labels_json}

In [18]:
bm25_space_result = {}

In [19]:
for cnt, ans in enumerate(answer):
    bm25_space_result[test_question_ids[cnt]] = []
    for a in ans:
        bm25_space_result[test_question_ids[cnt]].append(test_document_ids[a])

In [24]:
test_label_clean = {}
for test_label in test_labels_json.keys():
    for test in test_labels_json[test_label]:
        test_label_clean[test] = test_label

In [32]:
mrr = 0

for clean in test_label_clean.keys():
    try:
        mrr += 1 / (bm25_space_result[clean].index(test_label_clean[clean]) + 1)
    except:
        pass
    
print(mrr / len(test_label_clean))

0.7815034032065364


In [43]:
with open('bm25_space_result.json', 'w') as f:
    json.dump(bm25_space_result, f)

In [34]:
len(test_labels_json)

24711

In [35]:
len(test_question)

46691

In [38]:
for t in train_labels_json.keys():
    print(train_labels_json[t])
    break

['QUES_TNC71lb33r', 'QUES_TP1ywlo2lO']


In [39]:
train_question_ids = [t for key in train_labels_json.keys() for t in train_labels_json[key]]

In [42]:
len(train_question_ids)

186430